In [1]:
import pyscf
import pyscf.tools

from orbitalpartitioning import *

In [2]:
molecule = """
He 0.00 0.00 0.00
He 1.5 0.00 0.00
He 1.5 1.5 0.00
He 0.00 1.5 0.00
He 0.70 0.70 2.20
He 2.20 0.70 2.20
He 2.20 2.20 2.20
He 0.70 2.20 2.20
"""

basis = "6-31g"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   0, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  C2h


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 8  num. singly occ = 0
init E= -23.1076217221877
HOMO (Bu) = -0.688618622209975  LUMO (Ag) = 1.28511974158208
cycle= 1 E= -22.6861987184972  delta_E= 0.421  |g|= 0.0323  |ddm|= 0.572
HOMO (Bu) = -0.777084850059619  LUMO (Ag) = 1.28018555035836
cycle= 2 E= -22.6864405980352  delta_E= -0.000242  |g|= 0.00402  |ddm|= 0.0148
HOMO (Bu) = -0.776241389340644  LUMO (Ag) = 1.2808600917534
cycle= 3 E= -22.686444423719  delta_E= -3.83e-06  |g|= 5.71e-05  |ddm|= 0.00224
HOMO (Bu) = -0.776271438604529  LUMO (Ag) = 1.2808282002111
cycle= 4 E= -22.68644442446

In [3]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        frag1.append(ao_idx)
        full.append(ao_idx)
    elif ao[0] == 1:
        frag1.append(ao_idx)
        full.append(ao_idx)
    elif ao[0] == 2:
        frag1.append(ao_idx)
        full.append(ao_idx)
    elif ao[0] == 3:
        frag1.append(ao_idx)
        full.append(ao_idx)
    elif ao[0] == 4:
        frag2.append(ao_idx)
        full.append(ao_idx)
    elif ao[0] == 5:
        frag2.append(ao_idx)
        full.append(ao_idx)
    elif ao[0] == 6:
        frag2.append(ao_idx)
        full.append(ao_idx)
    elif ao[0] == 7:
        frag2.append(ao_idx)
        full.append(ao_idx)
            

frags = [frag1, frag2]
print(frags)

[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15]]


In [4]:
# Define projectors
X = np.eye(nbas) 
X = scipy.linalg.sqrtm(S)
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])

In [5]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact,Vact))

 Partition   16 orbitals into a total of   16 orbitals
            Index   Sing. Val. Space       
                0   2.23216799            2*
                1   2.15957480            2*
                2   1.78865553            2*
                3   1.77100106            2*
                4   1.71371725            2*
                5   1.69279266            2*
                6   1.50355282            2*
                7   1.42400527            2*
                8   0.91010547            0*
                9   0.90574412            0*
               10   0.80195521            0*
               11   0.79999855            0*
               12   0.79310281            0*
               13   0.79086942            0*
               14   0.70335622            0*
               15   0.68704195            0*


In [13]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1



for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])
print(" init_fspace: ", init_fspace)
print(" clusters   : ", clusters)


 Fragment:  [0, 1, 2, 3, 4, 5, 6, 7]
 Partition   16 orbitals into a total of    8 orbitals
            Index   Sing. Val. Space       
                0   2.18292600            2*
                1   1.72741823            2*
                2   1.72306748            2*
                3   1.44288923            2*
                4   0.90801055            0*
                5   0.79624443            0*
                6   0.79619099            0*
                7   0.69634295            0*
                8   0.19890315            0
                9   0.12575189            0
               10   0.12400658            0
               11   0.07795862            0
               12   0.00444743            2
               13   0.00239249            2
               14   0.00229409            2
               15   0.00102944            2

 Fragment:  [8, 9, 10, 11, 12, 13, 14, 15]
 Partition   16 orbitals into a total of    8 orbitals
            Index   Sing. Val. Space       
        

In [14]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(16, 0)
(16, 16)
(16, 16)


In [15]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [16]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [17]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

In [18]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)